# 0 Configuration

In [ ]:
config = {
    "overview": {

    },
    "fe": {
        "id": False, # true
        "volume": False,
        "order_purchase_timestamp": False, # linked with 'shipping_limit_date'
        "shipping_limit_date": False,
        "order_estimated_delivery_date": False,
        "price_agg": False, # false
        "last_order_datetime": False,
        "frequency": False
    },
    "model": {
        "rfm_raw": {
            "preparation": False,
            "n_cluster": False,
            "silhouette": False,
            "explain": False,
            "persona": False
        },

    },
    "pca": {
        "do": True,
        "elbow": True,
        "silhouette": False
    }
}

***
# 1 Dependencies import

## 1.0 Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler, MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score

from yellowbrick.cluster import SilhouetteVisualizer, KElbowVisualizer

from math import ceil

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

np.random.seed(0)

***
## 1.1 Features selection

In [ ]:
def get_categorical_features_name(dataset, split_by_unique_count=True, split_count=10):
    features_name = dataset.select_dtypes(["object", "bool"]).columns
    if split_by_unique_count:
        less_uniques = [feature_name for feature_name in features_name if dataset[feature_name].nunique() <= split_count]
        lot_uniques = features_name.difference(less_uniques).tolist()
        return (less_uniques, lot_uniques)
    else:
        return features_name.values.tolist()

In [ ]:
def get_numerical_features_name(dataset):
    features_name = dataset.select_dtypes(["int64", "float64"]).columns.values.tolist()
    return features_name

In [ ]:
def progressiveFeatureSelection(df, n_clusters=3, max_features=4,):
    feature_list = list(df.columns)
    selected_features = list()
    # select starting feature
    initial_feature = ""
    high_score = 0
    for feature in feature_list:
        kmeans = KMeans(n_clusters=n_clusters, random_state=42)
        data_ = df[feature]
        labels = kmeans.fit_predict(data_.to_frame())
        score_ = silhouette_score(data_.to_frame(), labels)
        print("Proposed new feature {} with score {}". format(feature, score_))
        if score_ >= high_score:
            initial_feature = feature
            high_score = score_
    print("The initial feature is {} with a silhouette score of {}.".format(initial_feature, high_score))
    feature_list.remove(initial_feature)
    selected_features.append(initial_feature)
    for _ in range(max_features-1):
        high_score = 0
        selected_feature = ""
        print("Starting selection {}...".format(_))
        for feature in feature_list:
            selection_ = selected_features.copy()
            selection_.append(feature)
            kmeans = KMeans(n_clusters=n_clusters, random_state=42)
            data_ = df[selection_]
            labels = kmeans.fit_predict(data_)
            score_ = silhouette_score(data_, labels)
            print("Proposed new feature {} with score {}". format(feature, score_))
            if score_ > high_score:
                selected_feature = feature
                high_score = score_
        selected_features.append(selected_feature)
        feature_list.remove(selected_feature)
        print("Selected new feature {} with score {}". format(selected_feature, high_score))
    return selected_features

***
## 1.2 Pipeline construction

In [ ]:
def evaluate(dataset, model, scoring="neg_root_mean_squared_error"):

    ### DATASET PREPARATION ###

    # categorical_cols = get_categorical_features_name(dataset, split_by_unique_count=False)
    numerical_cols = get_numerical_features_name(dataset)

    # X = dataset[numerical_cols + categorical_cols]
    X = dataset[numerical_cols]

    ### PIPELINE CONSTRUCTION ###

    num_pipe = Pipeline(steps=[
        ("simple_imputer", SimpleImputer(strategy="mean")),
        ("minmax_scaler", StandardScaler())
    ])

    # cat_pipe = Pipeline(steps=[
    #     ("simple_imputer", SimpleImputer(strategy="most_frequent")),
    #     ("ordinal_encoder", OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=np.nan)),
    #     ("simple_imputer_bis", SimpleImputer(strategy="mean")),
    #     ("std_scaler", StandardScaler())
    # ])

    preprocessor = ColumnTransformer([
        ("num_pipe", num_pipe, numerical_cols),
        # ("cat_pipe", cat_pipe, categorical_cols)
    ])

    pipeline = Pipeline([
        ("transforms", preprocessor),
        ("model", model)
    ])

    preprocessor.fit(X)
    array_preproc = preprocessor.transform(X)
    data_preproc = pd.DataFrame(data=array_preproc, columns=X.columns)

    return pipeline, preprocessor, data_preproc

In [ ]:
def pca_preprocessor(dataset):
    num_cols = get_numerical_features_name(dataset)
    categ_cols, _ = get_categorical_features_name(dataset)

    num_pipe = Pipeline(steps=[
        ("imputer_01", SimpleImputer(strategy="mean")),
        ("scaler_01", StandardScaler())
    ])

    categ_pipe = Pipeline(steps=[
        ("imputer_01", SimpleImputer(strategy="most_frequent")),
        ("encoder_01", OneHotEncoder(handle_unknown="ignore"))
    ])

    preprocessor = ColumnTransformer([
        ("num_pipe", num_pipe, num_cols),
        ("categ_pipe", categ_pipe, categ_cols)
    ])

    return preprocessor, num_cols, categ_cols

In [ ]:
def preprocess_data(dataset):
    preprocessor, num_cols, categ_cols = pca_preprocessor(dataset)

    raw_preprocessed_data = preprocessor.fit_transform(dataset)

    if categ_cols:
        categ_cols_preprocessed = preprocessor.transformers_[1][1]["encoder_01"].get_feature_names_out(categ_cols).tolist()
    else:
        categ_cols_preprocessed = []

    col_names = num_cols + categ_cols_preprocessed

    preprocessed_data = pd.DataFrame(data=raw_preprocessed_data, columns=col_names)

    return preprocessed_data

***
## 1.3 Cluster evaluation

In [ ]:
def elbow_plot(range, data, figsize=(10,10)):
    intertia_list = []
    for n in range:
        kmeans = KMeans(n_clusters=n, random_state=1)
        kmeans.fit(data)
        intertia_list.append(kmeans.inertia_)
    fig = plt.figure(figsize=figsize)
    ax = fig.add_subplot(111)
    sns.lineplot(y=intertia_list, x=range, ax=ax)
    ax.set_xlabel("Cluster")
    ax.set_ylabel("Inertia")
    ax.set_xticks(list(range))
    fig.show()

In [ ]:
def silhouette_plot(range, data, n_cols=2, figsize=(20, 10), wspace=0.35, hspace=0.35):
    if range[0] == 0:
        raise Exception("Range must not include 0")

    n_rows = ceil(len(range)/n_cols)
    index = 0

    plt.subplots(n_rows, n_cols, figsize=figsize)
    plt.subplots_adjust(wspace=wspace, hspace=hspace)

    for n in range:
        kmeans = KMeans(n, random_state=1)

        plt.subplot(n_rows, n_cols, index+1)
        sv = SilhouetteVisualizer(kmeans, colors="yellowbrick")
        sv.fit(data)

        index += 1
    
    plt.show()

In [ ]:
def find_optimal_epsilon(n_neighbors, data):
    neigh = NearestNeighbors(n_neighbors=n_neighbors)
    nbrs = neigh.fit(data)
    distances, indices = nbrs.kneighbors(data)
    distances = np.sort(distances, axis=0)
    distances = distances[:,1]
    plt.plot(distances)

***
## 1.4 Dimensionality reduction

In [ ]:
def scree_plot(dataset, figsize=(15, 5)):
    pca = PCA()
    pca.fit(dataset)

    plt.figure(figsize=figsize)
    explain_variance = pd.Series(pca.explained_variance_ratio_)
    explain_variance.plot(kind="bar", alpha=0.7)

    total = 0
    var_ls = []
    for x in explain_variance:
        total = total + x
        var_ls.append(total)
    
    pd.Series(var_ls).plot(marker="o", alpha=0.7)
    plt.xlabel("Principle Components", fontsize="x-large")
    plt.ylabel("Percentage Variance Explained", fontsize="x-large")
    plt.title("Scree plot", fontsize="xx-large")
    plt.show()

    return pca

In [ ]:
def apply_pca(dataset, columns, n_components):
    pca = PCA(n_components=n_components)
    pca_data = pca.fit_transform(dataset[columns])
    components_name = [f"PC{i+1}" for i in range(pca_data.shape[1])]
    pca_data = pd.DataFrame(data=pca_data, columns=components_name)
    loadings = pd.DataFrame(data=pca.components_.T, columns=components_name, index=columns)
    return pca, pca_data, loadings

In [ ]:
def plot_variance(pca, width=8, dpi=100):
    fig, axs = plt.subplots(1, 2)
    n = pca.n_components_
    grid = np.arange(1, n + 1)
    evr = pca.explained_variance_ratio_

    axs[0].bar(grid, evr)
    axs[0].set(
        xlabel="Component", title="% Explained Variance", ylim=(0.0, 1.0)
    )

    cumulative_variance = np.cumsum(evr)
    axs[1].plot(np.r_[0, grid], np.r_[0, cumulative_variance], "o-")
    axs[1].set(
        xlabel="Component", title="% Cumulative Variance", ylim=(0.0, 1.0)
    )

    fig.set(figwidth=8, dpi=100)
    return axs

***
## 1.5 RFM

In [ ]:
def rfm_distplot(dataset, customer_id, figsize=(20, 5)):
    warnings.filterwarnings('ignore')
    dataset = dataset.loc[:, dataset.columns.difference(customer_id)]

    fig, axes = plt.subplots(1, 3, figsize=figsize)
    for i, feature in enumerate(dataset.columns):
        sns.distplot(dataset[feature], ax=axes[i])

In [ ]:
def correlation_heatmap(dataset, figsize=(30, 20)):
    plt.figure(figsize=figsize)

    correlation = dataset.corr()
    mask = np.triu(np.ones_like(correlation, dtype=bool))

    sns.heatmap(data=correlation, mask=mask, annot=True, vmax=.75, center=0,
                square=True, linewidths=.5, cbar_kws={"shrink": .5})

    plt.title("Correlation heatmap", size=20)
    plt.xticks(rotation=45, size=16, ha="right")
    plt.yticks(size=16)
    plt.show()

In [ ]:
def elbow(dataset, k=(2, 12)):
    model = KMeans(random_state=1)
    elbow_visualizer = KElbowVisualizer(model, k=k)

    elbow_visualizer.fit(dataset)
    elbow_visualizer.poof()

In [ ]:
def silhouette(dataset, n):
    model = KMeans(n, random_state=1)
    silhouette_visualizer = SilhouetteVisualizer(model)

    silhouette_visualizer.fit(dataset)
    silhouette_visualizer.poof()

In [ ]:
def explain(dataset, dataset_preproc):
    model = KMeans(n_clusters=4, random_state=1).fit(dataset_preproc)
    data_explain = dataset
    data_explain["Cluster"] = model.labels_

    data_explain_melt = pd.melt(data_explain, id_vars=["customer_id", "Cluster"], value_vars=["Récence", "Fréquence", "Montant"], var_name="Features", value_name="Value")
    sns.lineplot("Features", "Value", hue="Cluster", data=data_explain_melt)
    plt.legend()

    return data_explain.groupby("Cluster").agg({
    "Récence": ["mean", "min", "max"],
    "Fréquence": ["mean", "min", "max"],
    "Montant": ["mean", "min", "max", "count"]
})

***
# 1.6 Explainability

In [ ]:
def explain_global(dataset, dataset_preproc, clusters, figsize=(30, 10)):
    model = KMeans(n_clusters=clusters, random_state=1).fit(dataset_preproc)
    data_explain = dataset.copy()
    data_explain.loc[:, "Cluster"] = model.labels_

    data_explain_melt = pd.melt(data_explain, id_vars=["customer_id", "Cluster"], value_vars=dataset_preproc.columns, var_name="Features", value_name="Value")

    plt.figure(figsize=figsize)
    sns.lineplot(x="Features", y="Value", hue="Cluster", data=data_explain_melt)
    plt.legend()

    return data_explain

***
# 2 Data loading

In [ ]:
data = pd.read_csv("data/data_cleaned.csv", delimiter=",")

In [ ]:
# data = pd.read_csv("data/data_cleaned.csv", delimiter=",", parse_dates=["order_purchase_timestamp", "order_approved_at", "order_delivered_carrier_date", "order_delivered_customer_date", "order_estimated_delivery_date", "review_creation_date", "review_answer_timestamp", "shipping_limit_date"], infer_datetime_format=True)

***
# 3 Overview

In [ ]:
data.info()

In [ ]:
data.describe()

In [ ]:
data.describe(include="object")

***
# 4 Feeture engineering

## 4.0 Remove *_id

In [ ]:
def fe_id_remove(dataset, diff=[]):
    id_cols = dataset.columns[dataset.columns.str.contains("_id")]
    id_cols = id_cols.difference(diff)
    dataset.drop(columns=id_cols, inplace=True)
    return dataset

***
## 4.1 Volume

In [ ]:
def fe_volume(dataset):
    dataset["product_volume_cm3"] = dataset["product_length_cm"] * dataset["product_height_cm"] * dataset["product_width_cm"]
    return dataset

***
## 4.2 Order purchase timestamp

In [ ]:
def fe_purchase_timestamp(dataset):
    dataset.loc[:, "order_purchase_timestamp"] = dataset.loc[:, "order_purchase_timestamp"].apply(pd.to_datetime)
    opt = dataset.loc[:, "order_purchase_timestamp"]
    dataset["order_purchase_year"] = opt.dt.year
    dataset["order_purchase_month"] = opt.dt.month
    dataset["order_purchase_day"] = opt.dt.day
    dataset["order_purchase_hour"] = opt.dt.hour

    display = dataset.loc[:, ["order_purchase_timestamp", "order_purchase_year", "order_purchase_month", "order_purchase_day", "order_purchase_hour"]].head()
    return dataset

***
## 4.3 Shipping limit date

In [ ]:
def fe_shipping_date(dataset):
    dataset.loc[:, "shipping_limit_date"] = dataset.loc[:, "shipping_limit_date"].apply(pd.to_datetime)
    sld = data.loc[:, "shipping_limit_date"]
    dataset["shipping_limit_year"] = sld.dt.year
    dataset["shipping_limit_month"] = sld.dt.month
    dataset["shipping_limit_day"] = sld.dt.day
    dataset["shipping_limit_hour"] = sld.dt.hour

    display = dataset.loc[:, ["shipping_limit_date", "shipping_limit_year", "shipping_limit_month", "shipping_limit_day", "shipping_limit_hour"]].head()
    return dataset

***
## 4.4 Estimated delivery date

In [ ]:
def fe_delivery_date(dataset):
    dataset.loc[:, "order_estimated_delivery_date"] = dataset.loc[:, "order_estimated_delivery_date"].apply(pd.to_datetime)
    oedd = dataset.loc[:, "order_estimated_delivery_date"]
    dataset["order_estimated_delivery_year"] = oedd.dt.year
    dataset["order_estimated_delivery_month"] = oedd.dt.month
    dataset["order_estimated_delivery_day"] = oedd.dt.day
    dataset["order_estimated_delivery_hour"] = oedd.dt.hour

    display = dataset.loc[:, ["order_estimated_delivery_date", "order_estimated_delivery_year", "order_estimated_delivery_month", "order_estimated_delivery_day", "order_estimated_delivery_hour"]].head()
    return dataset

***
## 4.5 Price agg

In [ ]:
def fe_price(dataset):
    customer_price_agg = dataset.groupby("customer_id").agg({
        "price": ["min", "max", "mean", "sum"]
    })

    customer_price_agg.rename(columns={
        "min": "price_min",
        "max": "price_max",
        "mean": "price_mean",
        "sum": "price_sum"
    }, inplace=True)

    dataset = dataset.merge(customer_price_agg.price, on="customer_id")
    display = dataset.loc[:, ["customer_id", "price_min", "price_max", "price_mean", "price_sum"]].head()
    return dataset


***
## 4.6 Last order datetime

In [ ]:
def fe_recency(dataset):
    dataset.loc[:, "order_purchase_timestamp"] = dataset.loc[:, "order_purchase_timestamp"].apply(pd.to_datetime)
    data_end = max(dataset.loc[:, "order_purchase_timestamp"] + pd.Timedelta(days=1))

    opt_agg = dataset.groupby("customer_id").agg({
        "order_purchase_timestamp": lambda x: (data_end - max(x)).days
    })

    opt_agg.columns = ["recency"]
    opt_agg = opt_agg.reset_index()

    dataset = dataset.merge(opt_agg, on="customer_id")

    display = dataset.loc[:, ["customer_id", "order_purchase_timestamp", "recency"]].head()
    return dataset

***
## 4.7 Frequency

In [ ]:
def fe_frequency(dataset):
    frequency = dataset.groupby("customer_id").agg({
        "order_id": ["count"]
    })

    frequency = frequency.order_id.reset_index()
    frequency.columns = ["customer_id", "frequency"]
    
    dataset = dataset.merge(frequency, how="left", on="customer_id")
    display = dataset.loc[:, ["customer_id", "frequency"]].head()
    return dataset

***
# 5 Modelisation

## 5.1 RFM

### 5.1.1 Preparation

RFM: Récence (dat de la dernière commande), Fréquence (des commandes), Montant (de la dernière commande ou sur une période donnée)

In [ ]:
if config["model"]["rfm_raw"]["preparation"]:
    rfm_cols = ["customer_id","order_purchase_timestamp", "order_id", "price"]
    data_rfm = data.loc[:, rfm_cols]

In [ ]:
if config["model"]["rfm_raw"]["preparation"]:
    data_rfm = fe_id_remove(data_rfm, diff=["customer_id", "order_id", "product_id"])
    data_rfm = fe_recency(data_rfm)
    data_rfm = fe_price(data_rfm)
    data_rfm = fe_frequency(data_rfm)

In [ ]:
if config["model"]["rfm_raw"]["preparation"]:
    data_rfm.drop(columns=["price", "price_min", "price_max", "price_mean", "order_purchase_timestamp", "order_id"], inplace=True)
    data_rfm.rename(columns={"recency": "Récence", "price_sum": "Montant", "frequency": "Fréquence"}, inplace=True)

In [ ]:
display = None
if config["model"]["rfm_raw"]["preparation"]:
    display = data_rfm.describe()
display

In [ ]:
if config["model"]["rfm_raw"]["preparation"]:
    rfm_distplot(data_rfm, ["customer_id"])

In [ ]:
if config["model"]["rfm_raw"]["preparation"]:
    correlation_heatmap(data_rfm, figsize=(20, 5))

In [ ]:
if config["model"]["rfm_raw"]["preparation"]:

    model = KMeans(n_clusters=4)

    pipeline, preproc, data_preproc = evaluate(data_rfm, model)

In [ ]:
display = None
if config["model"]["rfm_raw"]["preparation"]:
    display = data_preproc.head()
display

In [ ]:
if config["model"]["rfm_raw"]["preparation"]:
    rfm_distplot(data_preproc, ["customer_id"])

***
### 5.1.2 Cluster N analysis

In [ ]:
if config["model"]["rfm_raw"]["n_cluster"]:
    elbow(data_preproc.copy(), (2, 12))

In [ ]:
if config["model"]["rfm_raw"]["silhouette"]:
    silhouette(data_preproc.copy(), 4)

Silhouette for n=4  
![alt text](plots/kmeans_4n.png "Silhouette for n=4")

***
### 5.1.3 Explaination

In [ ]:
display = None
if config["model"]["rfm_raw"]["explain"]:
    display = explain(data_rfm, data_preproc)
display

In [ ]:
if config["model"]["rfm_raw"]["explain"]:
    pred = KMeans(n_clusters=4, random_state=1).fit_predict(data_preproc)

In [ ]:
if config["model"]["rfm_raw"]["explain"]:
    plt.subplots(3, 3, figsize=(30, 15))
    plt.subplots_adjust(wspace=0.15, hspace=0.15)

    plt.subplot(3 ,3, 1)
    sns.scatterplot(x=data_rfm["Récence"], y=data_rfm["Récence"], hue=pred, palette="deep")

    plt.subplot(3 ,3, 2)
    sns.scatterplot(x=data_rfm["Récence"], y=data_rfm["Fréquence"], hue=pred, palette="deep")

    plt.subplot(3 ,3, 3)
    sns.scatterplot(x=data_rfm["Récence"], y=data_rfm["Montant"], hue=pred, palette="deep")

    plt.subplot(3 ,3, 4)
    sns.scatterplot(x=data_rfm["Fréquence"], y=data_rfm["Récence"], hue=pred, palette="deep")

    plt.subplot(3 ,3, 5)
    sns.scatterplot(x=data_rfm["Fréquence"], y=data_rfm["Fréquence"], hue=pred, palette="deep")

    plt.subplot(3 ,3, 6)
    sns.scatterplot(x=data_rfm["Fréquence"], y=data_rfm["Montant"], hue=pred, palette="deep")

    plt.subplot(3 ,3, 7)
    sns.scatterplot(x=data_rfm["Montant"], y=data_rfm["Récence"], hue=pred, palette="deep")

    plt.subplot(3 ,3, 8)
    sns.scatterplot(x=data_rfm["Montant"], y=data_rfm["Fréquence"], hue=pred, palette="deep")

    plt.subplot(3 ,3, 9)
    sns.scatterplot(x=data_rfm["Montant"], y=data_rfm["Montant"], hue=pred, palette="deep")

In [ ]:
if config["model"]["rfm_raw"]["explain"]:
    pca, pca_data, loadings = apply_pca(data_preproc, data_preproc.columns, 2)
    reduced_data = pca_data.to_numpy()
    model = KMeans(init="k-means++", n_clusters=4)
    model.fit(reduced_data)

    h = 0.02

    x_min = reduced_data[:, 0].min() - 1
    x_max = reduced_data[:, 0].max() + 1

    y_min = reduced_data[:, 1].min() - 1
    y_max = reduced_data[:, 1].max() + 1

    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))

    Z = model.predict(np.c_[xx.ravel(), yy.ravel()])

    Z = Z.reshape(xx.shape)
    plt.figure(1)
    plt.clf()
    plt.imshow(
        Z, interpolation="nearest",
        extent=(xx.min(), xx.max(), yy.min(), yy.max()),
        cmap=plt.cm.Paired,
        aspect="auto",
        origin="lower"
    )

    plt.plot(reduced_data[:, 0], reduced_data[:, 1], "k.", markersize=2)

    centroids = model.cluster_centers_
    plt.scatter(
        centroids[:, 0],
        centroids[:, 1],
        marker="x",
        s=169,
        linewidths=3,
        color="w",
        zorder=10
    )

    plt.title("K-means clustering, PCA reduced dataset")

    plt.xlim(x_min, x_max)
    plt.ylim(y_min, y_max)
    plt.xticks(())
    plt.yticks(())
    plt.show()

In [ ]:
if config["model"]["rfm_raw"]["explain"]:
    loadings

In [ ]:
if config["model"]["rfm_raw"]["explain"]:
    plot_variance(pca)

***
### 5.1.4 Persona

In [ ]:
if config["model"]["rfm_raw"]["persona"]:
    display

In [ ]:
if config["model"]["rfm_raw"]["persona"]:
    data_preproc["cluster"] = pred
    data_preproc["order_purchase_timestamp"] = data["order_purchase_timestamp"]
    data_preproc = fe_purchase_timestamp(data_preproc)

In [ ]:
if config["model"]["rfm_raw"]["persona"]:
    data_preproc.head()

In [ ]:
if config["model"]["rfm_raw"]["persona"]:
    plt.figure(figsize=(20, 10))
    sns.kdeplot(data=data_preproc, x="order_purchase_month", hue="cluster", shade=False, palette="deep")

In [ ]:
if config["model"]["rfm_raw"]["persona"]:
    data_preproc["product_category_name"] = data["product_category_name"]

In [ ]:
if config["model"]["rfm_raw"]["persona"]:
    data_preproc.head()

In [ ]:
if config["model"]["rfm_raw"]["persona"]:
    tmp = data_preproc.groupby("cluster").agg({
        "product_category_name": "value_counts"
    })

In [ ]:
if config["model"]["rfm_raw"]["persona"]:
    tmp.info()

In [ ]:
if config["model"]["rfm_raw"]["persona"]:
    tmp.head()

In [ ]:
if config["model"]["rfm_raw"]["persona"]:
    plt.figure(figsize=(30, 10))
    sns.countplot(data=data_preproc, x="product_category_name", hue="cluster", palette="deep")

#### Cluster 0 - Persona 1 - Anciens

Dernier achat: il y a plus de 1 an  
Nombre d'achats: 1.2  
Montant: 122€  
Population: 45 156 / 40.42%  

Actions:  
...


#### Cluster 1 - Persona 2 - Fidèle

Dernier achat: il y a 8 mois  
Nombre d'achats: 5.4  
Montant: 358€  
Population: 4 659 / 4.17%  


#### Cluster 2 - Persona 3 - Nouveaux

Dernier achat: il y a 4 mois  
Nombre d'achats: 1.2  
Montant: 122€  
Population: 60 284 / 53.96%  


#### Cluster 3 - Persona 4 - Fort potentiel

Dernier achat: il y a 8 mois  
Nombre d'achats: 1.7  
Montant: 819€  
Population: 1 606 / 1.43%  

Actions:  
Client à fort potentiel. Mise en place d'une action de fidélisation nécessaire.  

***
## 5.2 Global

In [ ]:
data.head()

In [ ]:
display = None

if config["pca"]["do"]:
    global_cols = data.select_dtypes(["int64", "float64"]).columns
    global_cols = global_cols.append(data.loc[:, ["customer_id"]].columns)

    global_data = data[global_cols]

    global_data = fe_price(global_data)
    global_data = fe_volume(global_data)

    global_data.drop(columns=["order_item_id", "review_score", "payment_sequential", "payment_installments", "freight_value", "product_name_lenght", "product_photos_qty", "product_length_cm", "product_height_cm", "product_width_cm"], inplace=True)

    global_data_preproc = preprocess_data(global_data)

    display = global_data_preproc.head()

display

In [ ]:
if config["pca"]["do"]:
    pca = scree_plot(global_data_preproc)

In [ ]:
if config["pca"]["elbow"]:
    elbow(global_data_preproc)

In [ ]:
if config["pca"]["silhouette"]:
    silhouette(global_data_preproc, 4)

In [ ]:
if config["pca"]["do"]:
    explain_data = explain_global(global_data, global_data_preproc, 6)